In [179]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import linregress

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, concatenate, Conv2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

In [ ]:
def load_5models_from_disk():
    models = []
    for i in range(5):
        json_file = open("./model" + str(i) +".json", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        loaded_model.load_weights("temp" + str(i) +".hdf5")
        print("Loaded model from disk")

        # evaluate loaded model on test data
        loaded_model.compile(optimizer='rmsprop',
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])
        models.append(loaded_model)
    return models

In [ ]:
def shrink_array(array,size):
    
    ratio = float(len(array)) / float(size+1)
    res = []
    for i in range(size):
        res.append(np.mean(array[math.floor(i*ratio):math.ceil((i+1.0)*ratio)], axis = 0))
    return np.array(res)

In [146]:
train_data = pd.DataFrame.from_csv("../../data/database/train_data.csv")
test_data = pd.DataFrame.from_csv("../../data/database/test_data.csv")

if 'SENSORTYPE' in train_data.columns:
    train_data = train_data[train_data.SENSORTYPE == 1]
    test_data = test_data[test_data.SENSORTYPE == 1]

In [159]:
feature_cols = ['d_1','d_2', 'd_3']
train_data.columns

Index(['unix_timestamp', 'TagName', 'tester_id', 'v_1', 'v_2', 'v_3', 'd_1',
       'd_2', 'd_3', 'global_acc1', 'global_acc2', 'global_acc3'],
      dtype='object')

In [147]:

global_acc_min = np.min(train_data[['global_acc1', 'global_acc2', 'global_acc3']].values)
global_acc_max = np.max(train_data[['global_acc1', 'global_acc2', 'global_acc3']].values)
v_min = np.min(train_data[['v_1', 'v_2', 'v_3']].values)
v_max = np.max(train_data[['v_1', 'v_2', 'v_3']].values)
d_min = np.min(train_data[['d_1','d_2', 'd_3']].values)
d_max = np.max(train_data[['d_1','d_2', 'd_3']].values)

In [160]:
def get_feature_label(data):
    groups = data.groupby(['TagName','tester_id'])
    keys = groups.groups.keys()
    y = []
    X = []

    for k in keys:
        frame_feature = shrink_array(groups.get_group(k)[feature_cols].values, 50)
        X.append(frame_feature)
        y.append(k[0])
    X = np.array(X)
    y = np.array(y)
    return X,y

In [187]:
def SVC_training(X_train, y_train,x_test, y_test):
    if len(X_train.shape) > 2:
        X_train = X_train.reshape(list(X_train.shape)[0],-1)
        x_test = x_test.reshape(list(x_test.shape)[0],-1)
    
    max_score = 0
    max_i = 0
    max_res = []
    for i in range(1,21):
        i = i/2
        clf4 = SVC(kernel='linear', C=i, degree=3, verbose = True)

        clf4.fit(X_train, y_train) 
        # joblib.dump(clf4, '../../Results/baseline SVC 0.80 raw data acc with gyro 200 chunk.pkl') 
        res = clf4.predict(x_test)
        score = accuracy_score(y_test, res)
        if score>max_score:
            max_score = score
            max_i = i
            max_res = res
    print("max score: " + str(max_score) + "C = " + str(max_i))
    print(classification_report(y_test, max_res))
    print(confusion_matrix(y_test, max_res))

def DL_training(X_train, y_train,x_test, y_test):

    y = np.concatenate([y_train,y_test])
    tag_list = []
    for i in range(10):
        tag_list.append(['Tag'+str(i),i])
    for i in tag_list:
        tag_str = i[0]
        tag_int = i[1]
        y[y==tag_str] = tag_int
    y_categorical = to_categorical(y)
    
    y_train_cate = y_categorical[:len(y_train)]
    y_test_cate = y_categorical[len(y_train):]

    X_train = X_train.reshape(list(X_train.shape)[0],-1)
    x_test = x_test.reshape(list(x_test.shape)[0],-1)
    
    for i in range(5):
        # This returns a tensor
        inputs = Input(shape=(X_train.shape[1:]))

        # a layer instance is callable on a tensor, and returns a tensor
        layer1 = Dense(64, activation='relu')(inputs)
        layer2 = Dense(128, activation='relu')(layer1)
        layer3 = Dense(64, activation='relu')(layer2)
        layer4 = Dense(32, activation='relu')(layer3)
        predictions = Dense(10, activation='softmax')(layer4)

        mcp = ModelCheckpoint("./temp" + str(i) + ".hdf5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
        model = Model(inputs=inputs, outputs=predictions)
#         print(model.summary())
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        model_his = model.fit(X_train, y_train_cate, batch_size=32, epochs=40, verbose = 0,
                              validation_data=(x_test, y_test_cate), callbacks = [mcp]
                             )  # starts training

        model_json = model.to_json()
        with open("./model" + str(i) +".json", "w") as json_file:
            json_file.write(model_json)
            
    models = load_5models_from_disk()
    scores = []
    tests = []
    predicts = []
    
    for model in models:
        res = model.predict(x_test)
        scores.append(accuracy_score(np.argmax(y_test_cate,1), np.argmax(res, 1)))
        tests += np.argmax(y_test_cate,1).tolist()
        predicts += np.argmax(res, 1).tolist()
    print(classification_report(tests, predicts))
    print(confusion_matrix(tests, predicts))
    
def CONV1d_training(X_train, y_train,x_test, y_test):
    y = np.concatenate([y_train,y_test])
    tag_list = []
    for i in range(10):
        tag_list.append(['Tag'+str(i),i])
    for i in tag_list:
        tag_str = i[0]
        tag_int = i[1]
        y[y==tag_str] = tag_int
    y_categorical = to_categorical(y)
    
    y_train_cate = y_categorical[:len(y_train)]
    y_test_cate = y_categorical[len(y_train):]

    
    for i in range(5):
        input_val1 = Input(shape=X_train.shape[1:])

        con1 = Conv1D(filters=30,kernel_size=10)(input_val1)
        max_pooling_1d_1 = MaxPooling1D(pool_size=2, strides=None, padding='valid')(con1)
        flat_1 = Flatten()(max_pooling_1d_1)
        layer2 = Dense(128, activation='relu')(flat_1)
        layer4 = Dense(32, activation='relu')(layer2)
        predictions = Dense(y_categorical.shape[-1], activation='softmax')(layer4)

        model = Model(inputs = input_val1, outputs=predictions)
#         print(model.summary())
        mcp = ModelCheckpoint("./temp" + str(i) + ".hdf5", monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
    
        model_his = model.fit(X_train, y_train_cate, batch_size=32, epochs=40, verbose = 0,
                              validation_data=(x_test, y_test_cate), callbacks = [mcp]
                             )  # starts training

        model_json = model.to_json()
        with open("./model" + str(i) +".json", "w") as json_file:
            json_file.write(model_json)
            
    models = load_5models_from_disk()
    scores = []
    tests = []
    predicts = []
    
    for model in models:
        res = model.predict(x_test)
        scores.append(accuracy_score(np.argmax(y_test_cate,1), np.argmax(res, 1)))
        tests += np.argmax(y_test_cate,1).tolist()
        predicts += np.argmax(res, 1).tolist()
    print(classification_report(tests, predicts))
    print(confusion_matrix(tests, predicts))

In [90]:
import matplotlib.pyplot as plt
def plot_gesture(gesture):
    for i in range(gesture.shape[1]):
        plt.plot(gesture[:,i])
    plt.show()

In [162]:
X_train, y_train = get_feature_label(train_data)
x_test, y_test = get_feature_label(test_data)


In [167]:
SVC_training(X_train, y_train, x_test, y_test)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]max score: 0.571428571429C = 8.0
             precision    recall  f1-score   support

       Tag0       0.52      0.62      0.57        21
       Tag1       0.37      0.71      0.48        21
       Tag2       0.62      0.38      0.47        21
       Tag3       0.76      0.62      0.68        21
       Tag4       0.80      0.95      0.87        21
       Tag5       0.53      0.38      0.44        21
       Tag6       0.62      0.62      0.62        21
       Tag7       0.47      0.38      0.42        21
       Tag8       0.53      0.38      0.44        21
       Tag9       0.67      0.67      0.67        21

avg / total       0.59      0.57      0.57       210

[[13  1  0  0  2  0  2  1  2  0]
 [ 0 15  1  1  0  1  0  3  0  0]
 [ 0  7  8  1  0  1  1  1  1  1]
 [ 0  5  0 13  0  2  0  0  0  1]
 [ 1  0  0  0 20  0  0  0  0  0]
 [ 

In [ ]:
DL_training(X_train, y_train, x_test, y_test)

Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
             precision    recall  f1-score   support

          0       0.39      0.33      0.36       105
          1       0.59      0.77      0.67       105
          2       0.48      0.42      0.45       105
          3       0.54      0.61      0.57       105
          4       0.79      0.80      0.80       105
          5       0.60      0.36      0.45       105
          6       0.42      0.47      0.44       105
          7       0.61      0.57      0.59       105
          8       0.49      0.40      0.44       105
          9       0.56      0.77      0.65       105

avg / total       0.55      0.55      0.54      1050

[[35  3  3  5  7  0 32  4  8  8]
 [ 0 81  2  1  0 10  0 10  1  0]
 [ 8 14 44 16  0  4  6  7  5  1]
 [ 4 12  5 64  0  1  1  2  4 12]
 [ 4  0  0  4 84  0  1  0  0 12]
 [ 2  3  3 17  7 38 11  0 17  7]
 [29  1  9  2  6  4 49  0  4  1]
 [ 0  9 10  3  

In [ ]:
CONV1d_training(X_train, y_train, x_test, y_test)

Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk
Loaded model from disk


In [ ]:
m